In [1]:
%load_ext autoreload

In [2]:
import sys

if '../' not in sys.path:
    sys.path.append('../')

if '../../keyhold' not in sys.path:
    sys.path.append('../../keyhold')

In [3]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from fractions import Fraction

In [4]:
%autoreload

from fxtrade.stock import Stock, Rate
from fxtrade.trade import Trade, TradePair, History, Report
from fxtrade.stocks import JPY, BTC

from fxtrade.interface.yfinance import YahooFinanceAPI
from fxtrade.interface.bitflyer import BitflyerAPI

from fxtrade.wallet import Wallet
from fxtrade.chart import Chart
from fxtrade.trader import Trader
from fxtrade.fx import FX

In [5]:
from keyhold import KeyHold

key_config = KeyHold()

In [6]:
# チャート取得のための API を指定
chart_api = YahooFinanceAPI(api_key=key_config['yahoo']['x-api-key'])

# チャートを管理するクラスの生成
chart = Chart(
            ticker='BTC-JPY',
            api=chart_api,
            data_dir='../data/chart',
        )

TypeError: Chart.__init__() got an unexpected keyword argument 'ticker'

In [ ]:
# 取引のための API を指定
trade_api = BitflyerAPI(api_key=key_config['bitflyer']['api_key'],
                        api_secret=key_config['bitflyer']['api_secret'])

#hist = trade_api.get_history(start_date=pd.Timestamp(2021, 12, 1))

# 取引を管理するクラスの生成
trader = Trader(
            api=trade_api,
            history=None,
            data_dir='../data/trader',
         )

In [ ]:
wallet = Wallet(['JPY', 'BTC'])
wallet

In [ ]:
# 取引アルゴリズムを統括するためのクラスを生成
fx = FX(wallet=wallet, chart=chart, trader=trader)
fx

In [ ]:
fx.chart.dirmap

In [ ]:
fx.wallet

In [ ]:
fx.sync_wallet(codes={'JPY', 'BTC'})

In [ ]:
fx.wallet

In [ ]:
fx

In [ ]:
hist = fx.trader.get_history(start_date=datetime(2022, 2, 1))

In [ ]:
hist.summarize().applymap(lambda x: x if not isinstance(x, Fraction) else float(x))

In [ ]:
trade = fx.get_max_available()
trade

In [ ]:
trade = fx.get_max_salable()
trade

In [ ]:
fx.get_last_trade()

In [ ]:
UPDATE = False

if UPDATE:
    fx.chart.update()
    fx.chart.save()
else:
    fx.chart.load()

In [ ]:
fx.chart.dfs

In [ ]:
fx = fx.create_emulator(root_dir='../data/emulator')

In [ ]:
fx.chart.dfs

In [ ]:
fx.chart.download('1mo-15m')

In [ ]:
fx.chart.flush()
fx.chart.dfs

# 確認

In [ ]:
#_ = fx.chart.update(t=datetime(2022, 7, 14, 17, 15))
_ = fx.chart.update(t=datetime(2022, 8, 20, 17, 15))

In [ ]:
fx.chart['10y-1d']

In [ ]:
fx.chart['5d-1m']

In [ ]:
fx.chart.normalize()

In [ ]:
%autoreload

from fxtrade.analysis import *

In [ ]:
chart = fx.chart.dfs['1mo-15m']

low = chart['low']
high = chart['high']
glow = emaverage(low, 0.87, pd.Timedelta(minutes=15))

In [ ]:
fx.chart.normalize('1mo-15m')

In [ ]:
df_a = analyze(fx.chart['1mo-15m']['low'])
df_a

In [ ]:
plot_laplace(df_a['diff'])

In [ ]:
estimate_probability(df_a['diff'], 1)

# 自動取引アルゴリズム

In [ ]:
%autoreload
from fxtrade.algorithm.tenet import calc_zpc, calc_best_alpha, Tenet

In [ ]:
fx.chart.normalize('1mo-15m')

df = analyze(fx.chart['1mo-15m']['low'])
df

In [ ]:
loc, scale = estimate_laplace_param(df['diff'].iloc[-100:])
loc, scale

In [ ]:
crange_interval = '1mo-15m'
dt = pd.Timedelta(minutes=15)

In [ ]:
tenet = Tenet(crange_interval=crange_interval, dt=dt, window=3, alpha=0.87)

In [ ]:
tenet.analyze(fx)

### 買い時
* アップトレンドかつ平均より下かつ暴落ではない
* アップトレンドかつ暴騰中（大きく張る）
* アップトレンドかつ暴落終了直後
* ダウントレンドかつ暴落終了後の暴騰（大きく張る）

### 売り時
* 暴騰終了直後
* 暴落開始

In [ ]:
tenet.plot()

In [ ]:
tenet.df_low.iloc[[-1]]

In [ ]:
trade = tenet(fx)
trade

In [ ]:
fx.apply(tenet)

In [ ]:
try:
    fx.order(trade)
except ValueError as e:
    print('No order was made:', e)

In [ ]:
trade

In [ ]:
fx.trader.chart

In [ ]:
#fx.sell(fx.get_max_salable())

In [ ]:
#fx.execute(tenet)

In [ ]:
#fx.buy(trade)

In [ ]:
#resp = fx.trader.api.buy(float(trade.y.q))
#resp

In [ ]:
# sel = tenet.sell(fx)

In [ ]:
# sel

In [ ]:
# fx.trader.api.sell(float(sel.x.q))